# **Data Praparation for Deep Learning**

### **Load Libraies**

In [ ]:
import numpy as np # linear algebra
import pickle
import datetime
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
#from tensorflow.keras.models import Sequential, Input
#from tensorflow.keras.utils import utils
# from sklearn.metrics import log_loss
from numpy.random import permutation
import sklearn.metrics as metrics
import keras
import keras.backend as K
import cv2
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import LabelEncoder
import os
import zipfile
import torch.nn as nn
from IPython.core.history import catch_corrupt_db
%matplotlib inline
# Any results you write to the current directory are saved as output.

### **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Unzip data**

In [ ]:
#zipsrc=zipfile.ZipFile("/content/drive/MyDrive/Community Help /Abuse Image classification/safe.zip",'r')
#zipsrc.extractall("/content/drive/MyDrive/Community Help /Abuse Image classification/")
#zipsrc.close()
#print("Unzip completed..")

Unzip completed..


### **Variables Declaration**

In [ ]:
img_train_path='/content/drive/MyDrive/Community Help /Abuse Image classification/'
img_test_path='/content/drive/MyDrive/Community Help /Abuse Image classification/Test/'
folderpath = '/content/drive/MyDrive/Community Help /Abuse Image classification/NumPyArrays/'
img_rows = 224
img_cols = 224

img_safe_paths=[]
img_sexy_paths=[]
img_nude_paths=[]

img_safe_labels=[]
img_sexy_labels=[]
img_nude_labels=[]

img_loads_gray=[]
img_loads_color=[]

img_safe_loads_resize_color=[]
img_sexy_loads_resize_color=[]
img_nude_loads_resize_color=[]

img_safe_loads_resize_gray=[]
img_sexy_loads_resize_gray=[]
img_nude_loads_resize_gray=[]


### **Only First time Run ( No need to run everytime )**

In [ ]:
Imagepath_safe="/content/drive/MyDrive/Community Help /Abuse Image classification/safe1/"
Imagepath_safe_files = os.listdir(Imagepath_safe)
print('Imagepath_safe_files Count : ',len(Imagepath_safe_files))
with open(r'/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_safe_files.txt', 'w') as fp:
    for item in Imagepath_safe_files:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Imagepath_safe_files Done')

Imagepath_safe_files Count :  34019
Imagepath_safe_files Done


In [ ]:
Imagepath_sexy="/content/drive/MyDrive/Community Help /Abuse Image classification/sexy/"
Imagepath_sexy_files = os.listdir(Imagepath_sexy)
print('Imagepath_sexy_files Count : ',len(Imagepath_sexy_files))
with open(r'/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_sexy_files.txt', 'w') as fp:
    for item in Imagepath_sexy_files:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Imagepath_sexy_files Done')

Imagepath_sexy_files Count :  38010
Imagepath_sexy_files Done


In [ ]:
Imagepath_nude="/content/drive/MyDrive/Community Help /Abuse Image classification/nude/"
Imagepath_nude_files = os.listdir(Imagepath_nude)
print('Imagepath_nude_files Count : ',len(Imagepath_nude_files))
with open(r'/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_nude_files.txt', 'w') as fp:
    for item in Imagepath_nude_files:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Imagepath_nude_files Done')

Imagepath_nude_files Count :  38000
Imagepath_nude_files Done


### **Load Images Files in to DataFrame**

In [ ]:
df_safe = pd.read_csv("/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_safe_files.txt",header=None)
df_safe["classname"]="safe1"
df_safe.columns=['img','classname']
print(df_safe.shape)

df_sexy = pd.read_csv("/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_sexy_files.txt",header=None)
df_sexy["classname"]="sexy"
df_sexy.columns=['img','classname']
print(df_sexy.shape)

df_nude = pd.read_csv("/content/drive/MyDrive/Community Help /Abuse Image classification/Imagepath_nude_files.txt",header=None)
df_nude["classname"]="nude"
df_nude.columns=['img','classname']
print(df_nude.shape)

(34019, 2)
(38010, 2)
(38000, 2)


In [ ]:
df_safe.head()

,img,classname
0,safe_5453.jpg,safe1
1,safe_5107.jpg,safe1
2,safe_5108.jpg,safe1
3,safe_5109.jpg,safe1
4,safe_511.jpg,safe1


In [ ]:
df_sexy.head()

,img,classname
0,sexy_9102.jpg,sexy
1,sexy_9103.jpg,sexy
2,sexy_9104.jpg,sexy
3,sexy_9105.jpg,sexy
4,sexy_9106.jpg,sexy


In [ ]:
df_nude.head()

,img,classname
0,nude_9099.jpg,nude
1,nude_91.jpg,nude
2,nude_910.jpg,nude
3,nude_9100.jpg,nude
4,nude_9101.jpg,nude


### **Utility methods**

In [ ]:
def load_color_image(path):
    img = cv2.imread(path)
    return img

In [ ]:
def load_color_resize_image(path, rows=None, cols=None, gray=True):
    img = cv2.imread(path)
    img_resize_color = cv2.resize(img,(rows,cols))
    return img_resize_color

In [ ]:
def load_gray_image(path, rows=None, cols=None, gray=True):
    img = cv2.imread(path,0)
    return img

In [ ]:
def load_gray_resize_image(path, rows=None, cols=None, gray=True):
    img = cv2.imread(path,0)
    img_resize_gray = cv2.resize(img,(rows,cols))
    return img_resize_gray

In [ ]:
def RecordLogs(logmessage,filename):
   file1 = open(r"/content/drive/MyDrive/Community Help /Abuse Image classification/"+filename, "w")  # append mode
   file1.write("%s\n" +logmessage)
   file1.close()

### **[  Safe Gray images Processing   ]**
##### Load Gray images with resize (224,224) and convert to numpy array  

In [ ]:
count=0;
for index, row in df_safe.iterrows():
    count= (count+1)
    imgname=img_train_path+str(row["classname"])+'/'+row["img"]
    RecordLogs("df_safe count : " +str(count),'Logs.txt')
    try:
      img_safe_loads_resize_gray.append(load_gray_resize_image(imgname,rows=img_rows, cols=img_cols))
    except:
      RecordLogs(imgname,'ImageErrorFiles_safe.txt')

RecordLogs("df_safe Total Processes count : " +str(count),'Logs.txt')

In [ ]:
len(img_safe_loads_resize_gray)

30056

In [ ]:
XsafeG = np.array(img_safe_loads_resize_gray).reshape(-1,img_rows, img_cols,1)

In [ ]:
np.save(folderpath+'img_safe_loads_resize_gray.npy',XsafeG)

### **[  Sexy Gray images Processing   ]**
##### Load Gray images with resize (224,224) and convert to numpy array  

In [ ]:
count=0;
for index, row in df_sexy.iterrows():
    count= (count+1)
    imgname=img_train_path+str(row["classname"])+'/'+row["img"]
    RecordLogs("df_sexy Current count : " +str(count),'Logs.txt')
    try:
      img_sexy_loads_resize_gray.append(load_gray_resize_image(imgname,rows=img_rows, cols=img_cols))
    except:
      RecordLogs(imgname,'ImageEr rorFiles_sexy.txt')
RecordLogs("df_sexy Total Processes count : " +str(count),'Logs.txt')

In [ ]:
len(img_sexy_loads_resize_gray)

38003

In [ ]:
XsexyG = np.array(img_sexy_loads_resize_gray).reshape(-1,img_rows, img_cols,1)

In [ ]:
np.save(folderpath+'img_sexy_loads_resize_gray.npy',XsexyG)

### **[  Nude Gray images Processing  ]**
##### Load Gray images with resize (224,224) and convert to numpy array  

In [ ]:
df_nude.iloc[12150:, :].head()

,img,classname
12150,nude_32633.jpg,nude
12151,nude_32634.jpg,nude
12152,nude_32635.jpg,nude
12153,nude_32636.jpg,nude
12154,nude_32637.jpg,nude


In [ ]:
count=0;
for index, row in df_nude.iloc[12150:, :].iterrows():
    count= (count+12147)
    imgname=img_train_path+str(row["classname"])+'/'+row["img"]
    RecordLogs("df_nude count : " +str(count),'Logs.txt')
    print(len(img_nude_loads_resize_gray))
    try:
      img_nude_loads_resize_gray.append(load_gray_resize_image(imgname,rows=img_rows, cols=img_cols))
    except:
      print(imgname)
RecordLogs("df_nude Total Processes count : " +str(count),'Logs.txt')

Streaming output truncated to the last 5000 lines.
32997
32998
32999
33000
33001
33002
33003
33004
33005
33006
33007
33008
33009
33010
33011
33012
33013
33014
33015
33016
33017
33018
33019
33020
33021
33022
33023
33024
33025
33026
33027
33028
33029
33030
33031
33032
33033
33034
33035
33036
33037
33038
33039
33040
33041
33042
33043
33044
33045
33046
33047
33048
33049
33050
33051
33052
33053
33054
33055
33056
33057
33058
33059
33060
33061
33062
33063
33064
33065
33066
33067
33068
33069
33070
33071
33072
33073
33074
33075
33076
33077
33078
33079
33080
33081
33082
33083
33084
33085
33086
33087
33088
33089
33090
33091
33092
33093
33094
33095
33096
33097
33098
33099
33100
33101
33102
33103
33104
33105
33106
33107
33108
33109
33110
33111
33112
33113
33114
33115
33116
33117
33118
33119
33120
33121
33122
33123
33124
33125
33126
33127
33128
33129
33130
33131
33132
33133
33134
33135
33136
33137
33138
33139
33140
33141
33142
33143
33144
33145
33146
33147
33148
33149
33150
33151
33152
33153
33154
3

In [ ]:
len(img_nude_loads_resize_gray)

37997

In [ ]:
XnudeG = np.array(img_nude_loads_resize_gray).reshape(-1,img_rows, img_cols,1)

In [ ]:
np.save(folderpath+'img_nude_loads_resize_gray.npy',XnudeG)

### **[  Safe Color images Processing  ]**
##### Load Color images with resize (224 ,224) and convert to numpy array

In [ ]:
df_safe.iloc[0:5001, :].tail()

,img,classname
4996,safe_36975.jpg,safe1
4997,safe_36976.jpg,safe1
4998,safe_36977.jpg,safe1
4999,safe_36978.jpg,safe1
5000,safe_35178.jpg,safe1


In [ ]:
df_safe.iloc[10001:15001, :].head()

,img,classname
10001,safe_32440.jpg,safe1
10002,safe_32441.jpg,safe1
10003,safe_32442.jpg,safe1
10004,safe_32443.jpg,safe1
10005,safe_32444.jpg,safe1


In [ ]:
len(df_safe)

34019

In [ ]:
df_safe.iloc[30001:, :].shape

(4018, 2)

In [ ]:
from multiprocessing import Process

def ProcesssafeColorImg(thread):
  if thread[0]=="1":
    df=df_safe.iloc[0:5001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="2":
    df=df_safe.iloc[5001:10001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="3":
    df=df_safe.iloc[10001:15001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="4":
    df=df_safe.iloc[15001:20001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="5":
    df=df_safe.iloc[20001:25001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="6":
    df=df_safe.iloc[25001:30001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="7":
    df=df_safe.iloc[30001:, :]
    print(thread [0]+" Started ...")


  for index, row in df.iterrows():
      #count= (count+1)
      imgname=img_train_path+str(row["classname"])+'/'+row["img"]
      #RecordLogs("df_safe Color count : " +str(count),'Logs.txt')
      print(thread[0]+" - img_safe_loads_resize_color  : " + str(len(img_safe_loads_resize_color)))
      try:
        img_safe_loads_resize_color.append(load_color_resize_image(imgname,rows=img_rows, cols=img_cols))
      except:
        print(imgname)
  XsafeC = np.array(img_safe_loads_resize_color).reshape(-1,img_rows, img_cols,3)
  if thread[0]=="1":
    np.save(folderpath+'img_safe_loads_resize_color1.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color1 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="2":
    np.save(folderpath+'img_safe_loads_resize_color2.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color2 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="3":
    np.save(folderpath+'img_safe_loads_resize_color3.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color3 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="4":
    np.save(folderpath+'img_safe_loads_resize_color4.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color4 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="5":
    np.save(folderpath+'img_safe_loads_resize_color5.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color5 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="6":
    np.save(folderpath+'img_safe_loads_resize_color6.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color6 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))
  if thread[0]=="7":
    np.save(folderpath+'img_safe_loads_resize_color7.npy',XsafeC)
    print(thread[0]+" img_safe_loads_resize_color7 Saved success")
    print(thread[0]+" : "+ str(len(img_safe_loads_resize_color)))

  #RecordLogs("df_safe Color Total count : " + str(count),'Logs.txt')


p = Process(target=ProcesssafeColorImg, args=(["1"]))
p.start()

p1 = Process(target=ProcesssafeColorImg,  args=(["2"]))
p1.start()

p2 = Process(target=ProcesssafeColorImg,  args=(["3"]))
p2.start()

p3 = Process(target=ProcesssafeColorImg, args=(["4"]))
p3.start()

p4 = Process(target=ProcesssafeColorImg, args=(["5"]))
p4.start()

p5 = Process(target=ProcesssafeColorImg, args=(["6"]))
p5.start()

p6 = Process(target=ProcesssafeColorImg, args=(["7"]))
p6.start()

p.join()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()


Streaming output truncated to the last 5000 lines.
1 - img_safe_loads_resize_color  : 3745
4 - img_safe_loads_resize_color  : 37606 - img_safe_loads_resize_color  : 3773
1 - img_safe_loads_resize_color  : 37462 - img_safe_loads_resize_color  : 3726
5 - img_safe_loads_resize_color  : 3720


3 - img_safe_loads_resize_color  : 3777
1 - img_safe_loads_resize_color  : 3747
4 - img_safe_loads_resize_color  : 3761
5 - img_safe_loads_resize_color  : 37212 - img_safe_loads_resize_color  : 37273 - img_safe_loads_resize_color  : 37786 - img_safe_loads_resize_color  : 3774


/content/drive/MyDrive/Community Help /Abuse Image classification/safe1/safe_13794.jpg

6 - img_safe_loads_resize_color  : 37743 - img_safe_loads_resize_color  : 37792 - img_safe_loads_resize_color  : 37281 - img_safe_loads_resize_color  : 3748

4 - img_safe_loads_resize_color  : 3762
6 - img_safe_loads_resize_color  : 3775
3 - img_safe_loads_resize_color  : 3780


5 - img_safe_loads_resize_color  : 3722
2 - img_safe_loads_res

In [ ]:
X1=np.load(folderpath+'img_safe_loads_resize_color1.npy')
X2=np.load(folderpath+'img_safe_loads_resize_color2.npy')
X3=np.load(folderpath+'img_safe_loads_resize_color3.npy')
X4=np.load(folderpath+'img_safe_loads_resize_color4.npy')
X5=np.load(folderpath+'img_safe_loads_resize_color5.npy')
X6=np.load(folderpath+'img_safe_loads_resize_color6.npy')
X7=np.load(folderpath+'img_safe_loads_resize_color7.npy')
X8=np.load(folderpath+'img_safe_loads_resize_gray.npy')
print("1. "+str(len(X1)))
print("2. "+str(len(X2)))
print("3. "+str(len(X3)))
print("4. "+str(len(X4)))
print("5. "+str(len(X5)))
print("6. "+str(len(X6)))
print("7. "+str(len(X7)))
print("Total. "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))

print("8. "+str(len(X8)))

1. 4406
2. 4428
3. 4441
4. 4395
5. 4434
6. 4416
7. 3536
Total. 30056
8. 30056


### **[  Sexy Color images Processing  ]**
##### Load Color images with resize (224 ,224) and convert to numpy array

In [ ]:
from multiprocessing import Process
def ProcesssexyColorImg(thread):
  if thread[0]=="1":
    df=df_sexy.iloc[0:5001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="2":
    df=df_sexy.iloc[5001:10001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="3":
    df=df_sexy.iloc[10001:15001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="4":
    df=df_sexy.iloc[15001:20001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="5":
    df=df_sexy.iloc[20001:25001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="6":
    df=df_sexy.iloc[25001:30001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="7":
    df=df_sexy.iloc[30001:, :]
    print(thread [0]+" Started ...")


  for index, row in df.iterrows():
      #count= (count+1)
      imgname=img_train_path+str(row["classname"])+'/'+row["img"]
      #RecordLogs("df_sexy Color count : " +str(count),'Logs.txt')
      print(thread[0]+" - img_sexy_loads_resize_color  : " + str(len(img_sexy_loads_resize_color)))
      try:
        img_sexy_loads_resize_color.append(load_color_resize_image(imgname,rows=img_rows, cols=img_cols))
      except:
        print(imgname)
  XsexyC = np.array(img_sexy_loads_resize_color).reshape(-1,img_rows, img_cols,3)
  if thread[0]=="1":
    np.save(folderpath+'img_sexy_loads_resize_color1.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color1 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="2":
    np.save(folderpath+'img_sexy_loads_resize_color2.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color2 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="3":
    np.save(folderpath+'img_sexy_loads_resize_color3.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color3 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="4":
    np.save(folderpath+'img_sexy_loads_resize_color4.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color4 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="5":
    np.save(folderpath+'img_sexy_loads_resize_color5.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color5 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="6":
    np.save(folderpath+'img_sexy_loads_resize_color6.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color6 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))
  if thread[0]=="7":
    np.save(folderpath+'img_sexy_loads_resize_color7.npy',XsexyC)
    print(thread[0]+" img_sexy_loads_resize_color7 Saved success")
    print(thread[0]+" : "+ str(len(img_sexy_loads_resize_color)))

  #RecordLogs("df_sexy Color Total count : " + str(count),'Logs.txt')


p = Process(target=ProcesssexyColorImg, args=(["1"]))
p.start()

p1 = Process(target=ProcesssexyColorImg,  args=(["2"]))
p1.start()

p2 = Process(target=ProcesssexyColorImg,  args=(["3"]))
p2.start()

p3 = Process(target=ProcesssexyColorImg, args=(["4"]))
p3.start()

p4 = Process(target=ProcesssexyColorImg, args=(["5"]))
p4.start()

p5 = Process(target=ProcesssexyColorImg, args=(["6"]))
p5.start()

p6 = Process(target=ProcesssexyColorImg, args=(["7"]))
p6.start()

p.join()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()


Streaming output truncated to the last 5000 lines.


2 - img_sexy_loads_resize_color  : 4667
3 - img_sexy_loads_resize_color  : 4724
6 - img_sexy_loads_resize_color  : 4766
1 - img_sexy_loads_resize_color  : 4708
4 - img_sexy_loads_resize_color  : 47337 - img_sexy_loads_resize_color  : 4756
5 - img_sexy_loads_resize_color  : 4667


3 - img_sexy_loads_resize_color  : 4725
4 - img_sexy_loads_resize_color  : 47342 - img_sexy_loads_resize_color  : 46681 - img_sexy_loads_resize_color  : 47097 - img_sexy_loads_resize_color  : 4757


6 - img_sexy_loads_resize_color  : 47677 - img_sexy_loads_resize_color  : 4758
6 - img_sexy_loads_resize_color  : 47685 - img_sexy_loads_resize_color  : 4668
4 - img_sexy_loads_resize_color  : 4735

3 - img_sexy_loads_resize_color  : 4726
5 - img_sexy_loads_resize_color  : 4669

2 - img_sexy_loads_resize_color  : 46691 - img_sexy_loads_resize_color  : 4710
6 - img_sexy_loads_resize_color  : 47694 - img_sexy_loads_resize_color  : 47363 - img_sexy_loads_resize_colo

In [ ]:
X1=np.load(folderpath+'img_sexy_loads_resize_color1.npy')
X2=np.load(folderpath+'img_sexy_loads_resize_color2.npy')
X3=np.load(folderpath+'img_sexy_loads_resize_color3.npy')
X4=np.load(folderpath+'img_sexy_loads_resize_color4.npy')
X5=np.load(folderpath+'img_sexy_loads_resize_color5.npy')
X6=np.load(folderpath+'img_sexy_loads_resize_color6.npy')
X7=np.load(folderpath+'img_sexy_loads_resize_color7.npy')
X8=np.load(folderpath+'img_sexy_loads_resize_gray.npy')
print("1. "+str(len(X1)))
print("2. "+str(len(X2)))
print("3. "+str(len(X3)))
print("4. "+str(len(X4)))
print("5. "+str(len(X5)))
print("6. "+str(len(X6)))
print("7. "+str(len(X7)))
print("Total. "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))

print("8. "+str(len(X8)))

1. 4996
2. 4998
3. 5000
4. 5000
5. 5000
6. 5000
7. 8009
Total. 38003
8. 38003


### **[  Nude Color images Processing  ]**
##### Load Color images with resize (224 ,224) and convert to numpy array

In [ ]:
from multiprocessing import Process
def ProcessnudeColorImg(thread):
  if thread[0]=="1":
    df=df_nude.iloc[0:5001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="2":
    df=df_nude.iloc[5001:10001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="3":
    df=df_nude.iloc[10001:15001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="4":
    df=df_nude.iloc[15001:20001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="5":
    df=df_nude.iloc[20001:25001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="6":
    df=df_nude.iloc[25001:30001, :]
    print(thread [0]+" Started ...")
  if thread[0]=="7":
    df=df_nude.iloc[30001:, :]
    print(thread [0]+" Started ...")


  for index, row in df.iterrows():
      #count= (count+1)
      imgname=img_train_path+str(row["classname"])+'/'+row["img"]
      #RecordLogs("df_nude Color count : " +str(count),'Logs.txt')
      print(thread[0]+" - img_nude_loads_resize_color  : " + str(len(img_nude_loads_resize_color)))
      try:
        img_nude_loads_resize_color.append(load_color_resize_image(imgname,rows=img_rows, cols=img_cols))
      except:
        print(imgname)
  XnudeC = np.array(img_nude_loads_resize_color).reshape(-1,img_rows, img_cols,3)
  if thread[0]=="1":
    np.save(folderpath+'img_nude_loads_resize_color1.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color1 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="2":
    np.save(folderpath+'img_nude_loads_resize_color2.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color2 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="3":
    np.save(folderpath+'img_nude_loads_resize_color3.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color3 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="4":
    np.save(folderpath+'img_nude_loads_resize_color4.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color4 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="5":
    np.save(folderpath+'img_nude_loads_resize_color5.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color5 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="6":
    np.save(folderpath+'img_nude_loads_resize_color6.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color6 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))
  if thread[0]=="7":
    np.save(folderpath+'img_nude_loads_resize_color7.npy',XnudeC)
    print(thread[0]+" img_nude_loads_resize_color7 Saved success")
    print(thread[0]+" : "+ str(len(img_nude_loads_resize_color)))

  #RecordLogs("df_nude Color Total count : " + str(count),'Logs.txt')


p = Process(target=ProcessnudeColorImg, args=(["1"]))
p.start()

p1 = Process(target=ProcessnudeColorImg,  args=(["2"]))
p1.start()

p2 = Process(target=ProcessnudeColorImg,  args=(["3"]))
p2.start()

p3 = Process(target=ProcessnudeColorImg, args=(["4"]))
p3.start()

p4 = Process(target=ProcessnudeColorImg, args=(["5"]))
p4.start()

p5 = Process(target=ProcessnudeColorImg, args=(["6"]))
p5.start()

p6 = Process(target=ProcessnudeColorImg, args=(["7"]))
p6.start()

p.join()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()


Streaming output truncated to the last 5000 lines.
1 - img_nude_loads_resize_color  : 4468
3 - img_nude_loads_resize_color  : 4495
5 - img_nude_loads_resize_color  : 45417 - img_nude_loads_resize_color  : 5766

7 - img_nude_loads_resize_color  : 5767
6 - img_nude_loads_resize_color  : 4858
4 - img_nude_loads_resize_color  : 4412
2 - img_nude_loads_resize_color  : 4475
3 - img_nude_loads_resize_color  : 4496
1 - img_nude_loads_resize_color  : 44695 - img_nude_loads_resize_color  : 4542

7 - img_nude_loads_resize_color  : 5768
6 - img_nude_loads_resize_color  : 4859
4 - img_nude_loads_resize_color  : 4413
2 - img_nude_loads_resize_color  : 4476
7 - img_nude_loads_resize_color  : 5769
1 - img_nude_loads_resize_color  : 4470
5 - img_nude_loads_resize_color  : 4543
2 - img_nude_loads_resize_color  : 4477
4 - img_nude_loads_resize_color  : 4414
5 - img_nude_loads_resize_color  : 4544
6 - img_nude_loads_resize_color  : 4860
7 - img_nude_loads_resize_color  : 5770
1 - img_nude_loads_resize_col

In [ ]:
X1=np.load(folderpath+'img_nude_loads_resize_color1.npy')
X2=np.load(folderpath+'img_nude_loads_resize_color2.npy')
X3=np.load(folderpath+'img_nude_loads_resize_color3.npy')
X4=np.load(folderpath+'img_nude_loads_resize_color4.npy')
X5=np.load(folderpath+'img_nude_loads_resize_color5.npy')
X6=np.load(folderpath+'img_nude_loads_resize_color6.npy')
X7=np.load(folderpath+'img_nude_loads_resize_color7.npy')
X8=np.load(folderpath+'img_nude_loads_resize_gray.npy')
print("1. "+str(len(X1)))
print("2. "+str(len(X2)))
print("3. "+str(len(X3)))
print("4. "+str(len(X4)))
print("5. "+str(len(X5)))
print("6. "+str(len(X6)))
print("7. "+str(len(X7)))
print("Total. "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))

print("8. "+str(len(X8)))

1. 5001
2. 5000
3. 5000
4. 5000
5. 5000
6. 5000
7. 7999
Total. 38000
8. 37997


### **Concatenate NumPy arrays**

In [ ]:
X1=np.load(folderpath+'img_safe_loads_resize_color1.npy')
X2=np.load(folderpath+'img_safe_loads_resize_color2.npy')
X3=np.load(folderpath+'img_safe_loads_resize_color3.npy')
X4=np.load(folderpath+'img_safe_loads_resize_color4.npy')
X5=np.load(folderpath+'img_safe_loads_resize_color5.npy')
X6=np.load(folderpath+'img_safe_loads_resize_color6.npy')
X7=np.load(folderpath+'img_safe_loads_resize_color7.npy')
print("1. img_safe_loads_resize_color1 "+str(len(X1)))
print("2. img_safe_loads_resize_color2 "+str(len(X2)))
print("3. img_safe_loads_resize_color3 "+str(len(X3)))
print("4. img_safe_loads_resize_color4 "+str(len(X4)))
print("5. img_safe_loads_resize_color5 "+str(len(X5)))
print("6. img_safe_loads_resize_color6 "+str(len(X6)))
print("7. img_safe_loads_resize_color7 "+str(len(X7)))
print("Total. Safe Color "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))

img_safe_loads_resize_color=np.concatenate((X1, X2,X3,X4,X5,X6,X7), axis=0)
np.save(folderpath+'img_safe_loads_resize_color.npy',img_safe_loads_resize_color)

1. img_safe_loads_resize_color1 4406
2. img_safe_loads_resize_color2 4428
3. img_safe_loads_resize_color3 4441
4. img_safe_loads_resize_color4 4395
5. img_safe_loads_resize_color5 4434
6. img_safe_loads_resize_color6 4416
7. img_safe_loads_resize_color7 3536
Total. Safe Color 30056


In [ ]:
X1=np.load(folderpath+'img_sexy_loads_resize_color1.npy')
X2=np.load(folderpath+'img_sexy_loads_resize_color2.npy')
X3=np.load(folderpath+'img_sexy_loads_resize_color3.npy')
X4=np.load(folderpath+'img_sexy_loads_resize_color4.npy')
X5=np.load(folderpath+'img_sexy_loads_resize_color5.npy')
X6=np.load(folderpath+'img_sexy_loads_resize_color6.npy')
X7=np.load(folderpath+'img_sexy_loads_resize_color7.npy')
print("1. img_sexy_loads_resize_color1 "+str(len(X1)))
print("2. img_sexy_loads_resize_color2 "+str(len(X2)))
print("3. img_sexy_loads_resize_color3 "+str(len(X3)))
print("4. img_sexy_loads_resize_color4 "+str(len(X4)))
print("5. img_sexy_loads_resize_color5 "+str(len(X5)))
print("6. img_sexy_loads_resize_color6 "+str(len(X6)))
print("7. img_sexy_loads_resize_color7 "+str(len(X7)))
print("Total. Sexy Color "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))


img_sexy_loads_resize_color=np.concatenate((X1, X2,X3,X4,X5,X6,X7), axis=0)
np.save(folderpath+'img_sexy_loads_resize_color.npy',img_sexy_loads_resize_color)




1. img_sexy_loads_resize_color1 4996
2. img_sexy_loads_resize_color2 4998
3. img_sexy_loads_resize_color3 5000
4. img_sexy_loads_resize_color4 5000
5. img_sexy_loads_resize_color5 5000
6. img_sexy_loads_resize_color6 5000
7. img_sexy_loads_resize_color7 8009
Total. Sexy Color 38003


In [ ]:
img_sexy_loads_resize_color=""
img_safe_loads_resize_color=""

In [ ]:
X1=np.load(folderpath+'img_nude_loads_resize_color1.npy')
X2=np.load(folderpath+'img_nude_loads_resize_color2.npy')
X3=np.load(folderpath+'img_nude_loads_resize_color3.npy')
X4=np.load(folderpath+'img_nude_loads_resize_color4.npy')
X5=np.load(folderpath+'img_nude_loads_resize_color5.npy')
X6=np.load(folderpath+'img_nude_loads_resize_color6.npy')
X7=np.load(folderpath+'img_nude_loads_resize_color7.npy')
print("1. img_nude_loads_resize_color1 "+str(len(X1)))
print("2. img_nude_loads_resize_color2 "+str(len(X2)))
print("3. img_nude_loads_resize_color3 "+str(len(X3)))
print("4. img_nude_loads_resize_color4 "+str(len(X4)))
print("5. img_nude_loads_resize_color5 "+str(len(X5)))
print("6. img_nude_loads_resize_color6 "+str(len(X6)))
print("7. img_nude_loads_resize_color7 "+str(len(X7)))
print("Total. nude Color "+str((len(X1)+len(X2)+len(X3)+len(X4)+len(X5)+len(X6)+len(X7))))

img_nude_loads_resize_color=np.concatenate((X1, X2,X3,X4,X5,X6,X7), axis=0)
np.save(folderpath+'img_nude_loads_resize_color.npy',img_nude_loads_resize_color)

1. img_nude_loads_resize_color1 5001
2. img_nude_loads_resize_color2 5000
3. img_nude_loads_resize_color3 5000
4. img_nude_loads_resize_color4 5000
5. img_nude_loads_resize_color5 5000
6. img_nude_loads_resize_color6 5000
7. img_nude_loads_resize_color7 7999
Total. nude Color 38000


In [ ]:
img_nude_loads_resize_color=""

In [ ]:
img_safe_loads_resize_color=np.load(folderpath+'img_safe_loads_resize_color.npy')
img_sexy_loads_resize_color=np.load(folderpath+'img_sexy_loads_resize_color.npy')
img_nude_loads_resize_color=np.load(folderpath+'img_nude_loads_resize_color.npy')
img_nude_loads_resize_color=img_nude_loads_resize_color[3:]
img_loads_resize_color=np.concatenate((img_safe_loads_resize_color, img_sexy_loads_resize_color,img_nude_loads_resize_color), axis=0)
np.save(folderpath+'img_loads_resize_color.npy',img_loads_resize_color)

In [ ]:
print("1. img_safe_loads_resize_color "+ str(len(img_safe_loads_resize_color)))
print("2. img_sexy_loads_resize_color "+ str(len(img_sexy_loads_resize_color)))
print("3. img_nude_loads_resize_color "+ str(len(img_nude_loads_resize_color)))

1. img_safe_loads_resize_color 30056
2. img_sexy_loads_resize_color 38003
3. img_nude_loads_resize_color 37997


In [ ]:
img_safe_loads_resize_color=""
img_sexy_loads_resize_color=""
img_nude_loads_resize_color=""
img_loads_resize_color=""

In [ ]:
img_safe_loads_resize_gray=np.load(folderpath+'img_safe_loads_resize_gray.npy')
print("Total. safe Gray "+ str(len(img_safe_loads_resize_gray)))
img_sexy_loads_resize_gray=np.load(folderpath+'img_sexy_loads_resize_gray.npy')
print("Total. sexy Gray "+ str(len(img_sexy_loads_resize_gray)))
img_nude_loads_resize_gray=np.load(folderpath+'img_nude_loads_resize_gray.npy')
print("Total. nude Gray "+ str(len(img_nude_loads_resize_gray)))

img_loads_resize_gray=np.concatenate((img_safe_loads_resize_gray, img_sexy_loads_resize_gray,img_nude_loads_resize_gray), axis=0)
np.save(folderpath+'img_loads_resize_gray.npy',img_loads_resize_gray)

Total. safe Gray 30056
Total. sexy Gray 38003
Total. nude Gray 37997


In [ ]:
img_safe_labels=[]
img_sexy_labels=[]
img_nude_labels=[]

for i in range(30056):
    img_safe_labels.append("safe")
for i in range(38003):
    img_sexy_labels.append("sexy")
for i in range(37997):
    img_nude_labels.append("nude")

In [ ]:
print("img_safe_labels : " + str(len(img_safe_labels)))
print("img_sexy_labels : " + str(len(img_sexy_labels)))
print("img_nude_labels : " + str(len(img_nude_labels)))

img_safe_labels : 30056
img_sexy_labels : 38003
img_nude_labels : 37997


#### **Load Labels and convert to numpy array**

In [ ]:
np.save(folderpath+'img_safe_labels.npy',img_safe_labels)
np.save(folderpath+'img_sexy_labels.npy',img_sexy_labels)
np.save(folderpath+'img_nude_labels.npy',img_nude_labels)

In [ ]:
img_labels=np.concatenate((img_safe_labels, img_sexy_labels,img_nude_labels), axis=0)
np.save(folderpath+'img_labels.npy',img_labels)

### **Convert Categorial to Classes**

In [ ]:
img_labels=np.load(folderpath+'img_labels.npy')

In [ ]:
lbls=[]
mapping = {'safe' : 0, 'sexy' :1, 'nude':2}
for val in img_labels:
  lbls.append(mapping[val])

In [ ]:
len(lbls)

106056

### **Read numpy array for Gray  images**
##### Split data into Tran , Val , Test set for Gray Images

In [ ]:
XG=np.load(folderpath+'img_loads_resize_gray.npy')

In [ ]:
len(XG)

106056

In [ ]:
X , X_test , y ,y_test = train_test_split(XG , lbls , test_size=0.2, random_state=0,stratify = lbls)

In [ ]:
X.shape  , X_test.shape , np.array(y).shape , np.array(y_test).shape

((84844, 224, 224, 1), (21212, 224, 224, 1), (84844,), (21212,))

In [ ]:
train_imgs , val_imgs , train_y , val_y = train_test_split( X, y ,test_size=0.1, random_state=0,stratify = y)

In [ ]:
train_imgs.shape , val_imgs.shape , np.array(train_y).shape ,  np.array(val_y).shape

((76359, 224, 224, 1), (8485, 224, 224, 1), (76359,), (8485,))

In [ ]:
XG=""

In [ ]:
np.save(folderpath+'train_imgs_gray.npy', train_imgs)
np.save(folderpath+'train_y_gray.npy', train_y)

In [ ]:
train_imgs=""
train_y=""

In [ ]:
np.save(folderpath+'val_imgs_gray.npy', val_imgs)
np.save(folderpath+'val_y_gray.npy', val_y)

In [ ]:
val_imgs=""
val_y=""

In [ ]:
np.save(folderpath+'X_test_gray.npy', X_test)
np.save(folderpath+'y_test_gray.npy', y_test)

In [ ]:
X_test=""
y_test=""

In [ ]:
np.save(folderpath+'X_gray.npy', X)
np.save(folderpath+'y_gray.npy', y)

In [ ]:
X=""
y=""

### **Read numpy array for  Color images**

##### Split data into Tran , Val , Test set for Color Images

In [ ]:
XC=np.load(folderpath+'img_loads_resize_color.npy')

In [ ]:
X , X_test , y ,y_test = train_test_split(XC , lbls , test_size=0.2, random_state=0,stratify = lbls)

In [ ]:
XC=""

In [ ]:
X.shape  , X_test.shape , np.array(y).shape , np.array(y_test).shape

((84844, 224, 224, 3), (21212, 224, 224, 3), (84844,), (21212,))

In [ ]:
train_imgs , val_imgs , train_y , val_y = train_test_split( X, y ,test_size=0.1, random_state=0,stratify = y)

In [ ]:
train_imgs.shape , val_imgs.shape , np.array(train_y).shape ,  np.array(val_y).shape

((76359, 224, 224, 3), (8485, 224, 224, 3), (76359,), (8485,))

In [ ]:
np.save(folderpath+'train_imgs_color.npy', train_imgs)
np.save(folderpath+'train_y_color.npy', train_y)

In [ ]:
train_imgs=""
train_y=""

In [ ]:
np.save(folderpath+'val_imgs_color.npy', val_imgs)
np.save(folderpath+'val_y_color.npy', val_y)

In [ ]:
val_imgs=""
val_y=""

In [ ]:
np.save(folderpath+'X_color.npy', X)
np.save(folderpath+'y_color.npy', y)

In [ ]:
X=""
y=""

In [ ]:
np.save(folderpath+'X_test_color.npy', X_test)
np.save(folderpath+'y_test_color.npy', y_test)

In [ ]:
X_test=""
y_test=""

# **Data Praparation for Machine Learning**

### **Load Libraies**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!pip install opencv-python==4.4.0.44
#!pip install opencv-contrib-python==4.4.0.44

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
import joblib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px

In [ ]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report
from skimage.feature import local_binary_pattern,hog
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
print (cv2 .__version__)

4.7.0


### **Features Extration**

In [ ]:
arraypath = '/content/gdrive/MyDrive/Community Help /Abuse Image classification/NumPyArrays/'

In [ ]:
def save_feature(feature, name):
  np.save(arraypath + name  +'.npy',feature)
def load_feature_nmpy(feature,name):
  np.load(arraypath + name  +'.npy')

#### HOG

In [ ]:
def get_hog(images, name='hog', save=False):
    result = np.array([hog(img, block_norm='L2') for img in images])
    if save:
        save_feature(result, name)
    return result

#### LBP

In [ ]:
def get_lbp(images, name='lbp', save=False):
  result = np.array([local_binary_pattern(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), 10, 3).flatten() for img in images])
  if save:
      save_feature(result, name)

  return result

#### SIFT

In [ ]:
def get_sift(images, name='sift', save=False):
    # SIFT descriptor for 1 image
    def get_image_sift(image, vector_size=32):
        alg = cv2.SIFT_create()
        kps = alg.detect(image, None)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # Making descriptor of same size
        # Descriptor vector size is 128
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)

        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        return dsc
    # SIFT descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_sift(img)
        features.append(dsc)
    result = np.array(features)
    if save:
        save_feature(result, name)
    return result

#### Color Histogram

In [ ]:
def get_color_hist(images, name='color_hist', save=False):
    histograms = []
    for img in images:
        histograms.append(cv2.calcHist([img], [0, 1, 2],None, [8,8,8], [0, 256, 0, 256, 0, 256]).flatten())
    result = np.array(histograms)
    if save:
        save_feature(result, name)
    return result

#### KAZE

In [ ]:
def get_kaze(images, name='kaze', save=False):
    def get_image_kaze(image, vector_size=32):
        alg = cv2.KAZE_create()
        kps = alg.detect(image)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)
        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        return dsc
    # KAZE descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_kaze(img)
        features.append(dsc)
    result = np.array(features)
    if save:
        save_feature(result, name)
    return result

### **Normalize Features**

In [ ]:
def norm_features_minmax(train, test):
  min_max_scaler = preprocessing.MinMaxScaler()
  norm_train = min_max_scaler.fit_transform(train)
  norm_test = min_max_scaler.transform(test)
  return norm_train, norm_test

In [ ]:
def norm_features_zscore(train, test):
    min_max_scaler = preprocessing.StandardScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_test = min_max_scaler.transform(test)
    return norm_train, norm_test

In [ ]:
def norm_features_minmax2(train,val,test):
    min_max_scaler = preprocessing.MinMaxScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_val = min_max_scaler.transform(val)
    norm_test = min_max_scaler.transform(test)
    return norm_train,norm_val, norm_test

In [ ]:
def norm_features_zscore2(train,val, test):
    min_max_scaler = preprocessing.StandardScaler()
    norm_train = min_max_scaler.fit_transform(train)
    norm_val = min_max_scaler.transform(val)
    norm_test = min_max_scaler.transform(test)
    return norm_train,norm_val,norm_test

In [ ]:
def flattened(images, color=cv2.COLOR_RGB2GRAY, name='flattened', save=False):
    """
    color: default RGB2GRAY, if None is passed then color is used as it is.
    """
    color_images = []
    if color is not None:
        for img in images:
            color_images.append(cv2.cvtColor(img, color))
    else:
        color_images = images

    count = len(color_images)

    result = np.array(color_images).reshape(count, -1)

    if save:
        save_feature(result, name)

    return result

In [ ]:
def combine_features(features, horizontal=True):
    if horizontal:
        return np.hstack(features)
    else:
        return np.vstack(features)

### **Load Data**

In [ ]:
train_imgs = np.load(arraypath+'train_imgs_gray.npy')
train_y = np.load(arraypath+'train_y_gray.npy')
val_imgs = np.load(arraypath+'val_imgs_gray.npy')
val_y = np.load(arraypath+'val_y_gray.npy')

X = np.load(arraypath+'X_gray.npy')
y = np.load(arraypath+'y_gray.npy')
X_test = np.load(arraypath+'X_test_gray.npy')
y_test = np.load(arraypath+'y_test_gray.npy')

In [ ]:
X.shape  , X_test.shape , np.array(y).shape ,  np.array(y_test).shape

((84844, 224, 224, 3), (21212, 224, 224, 3), (84844,), (21212,))

In [ ]:
train_imgs.shape , val_imgs.shape , np.array(train_y).shape , np.array(val_y).shape

((76359, 224, 224, 3), (8485, 224, 224, 3), (76359,), (8485,))

### **Training Features**

#### SIFT Features

In [ ]:
sift_train = get_sift(train_imgs, name='sift_train', save=False)
save_feature(sift_train, "sift_train")

ValueError: ignored

In [ ]:
sift_val = get_sift(val_imgs, name='sift_val', save=False)
save_feature(sift_val, "sift_val")

In [ ]:
sift_test = get_sift(X_test, name='sift_test', save=False)
save_feature(sift_test, "sift_test")

In [ ]:
sift_train = np.load(arraypath+'sift_train.npy')
sift_val = np.load(arraypath+'sift_val.npy')
sift_test = np.load(arraypath+'sift_test.npy')

In [ ]:
sift_train.shape,sift_val.shape,sift_test.shape

#### Color Histogram Features

In [ ]:
hist_train = get_color_hist(train_imgs, name='hist_train', save=False)
hist_val = get_color_hist(val_imgs, name='hist_val', save=False)
hist_test = get_color_hist(X_test, name='hist_val', save=False)

In [ ]:
save_feature(hist_train,'hist_train')
save_feature(hist_val,'hist_val')
save_feature(hist_test,'hist_test')

In [ ]:
hist_train = np.load(arraypath+'hist_train.npy')
hist_val = np.load(arraypath+'hist_val.npy')
hist_test = np.load(arraypath+'hist_test.npy')

In [ ]:
hist_train.shape, hist_val.shape, hist_test.shape

#### KAZE Features

In [ ]:
kaze_train = get_kaze(train_imgs, name='kaze_train', save=False)
kaze_val = get_kaze(val_imgs, name='kaze_val', save=False)
kaze_test = get_kaze(X_test, name='kaze_test', save=False)

In [ ]:
np.save(arraypath+'kaze_test.npy', kaze_test)
np.save(arraypath+'kaze_train.npy', kaze_train)
np.save(arraypath+'kaze_val.npy', kaze_val)

In [ ]:
kaze_train = np.load(arraypath+'kaze_train.npy')
kaze_val = np.load(arraypath+'kaze_val.npy')
kaze_test = np.load(arraypath+'kaze_test.npy')

In [ ]:
kaze_train.shape, kaze_val.shape, kaze_test.shape

#### HOG Features

In [ ]:
hog_train = get_hog(train_imgs, name='hog_train', save=False)
save_feature(hog_train, "hog_train")


In [ ]:
hog_val = get_hog(val_imgs, name='hog_val', save=False)
save_feature(hog_val, "hog_val")

In [ ]:
hog_test = get_hog(X_test, name='hog_test', save=False)
save_feature(hog_test, "hog_test")

In [ ]:
hog_train = np.load(arraypath+'hog_train.npy')
hog_val = np.load(arraypath+'hog_val.npy')
hog_test = np.load(arraypath+'hog_test.npy')

In [ ]:
hog_train.shape,hog_val.shape,hog_test.shape

#### LBP Features

In [ ]:
lbp_train = get_lbp(train_imgs, name='lbp_train', save=False)
save_feature(lbp_train,'lbp_train')

In [ ]:
lbp_val = get_lbp(val_imgs, name='lbp_val', save=False)
save_feature(lbp_val,'lbp_val')

In [ ]:
lbp_test = get_lbp(X_test, name='lbp_test', save=False)
save_feature(lbp_test,'lbp_test')

In [ ]:
lbp_train = np.load(arraypath+'lbp_train.npy')
lbp_val = np.load(arraypath+'lbp_val.npy')
lbp_test = np.load(arraypath+'lbp_test.npy')

In [ ]:
lbp_train.shape,lbp_val.shape,lbp_test.shape

### **Reduce Features by PCA**

#### HOG

In [ ]:
norm_hog_train, norm_hog_val,norm_hog_test = norm_features_minmax2(hog_train, hog_val,hog_test)

np.save(arraypath+'norm_hog_train.npy', norm_hog_train)
np.save(arraypath+'norm_hog_val.npy', norm_hog_val)
np.save(arraypath+'norm_hog_test.npy', norm_hog_test)

In [ ]:
norm_hog_train=np.load(arraypath+'norm_hog_train.npy')

In [ ]:
pca = PCA(n_components=100)
pca_hog_train = pca.fit_transform(norm_hog_train)

In [ ]:
np.save(arraypath+'pca_hog_train.npy', pca_hog_train)

In [ ]:
norm_hog_train=''
pca_hog_train=''

In [ ]:
norm_hog_val=np.load(arraypath+'norm_hog_val.npy')

In [ ]:
pca_hog_val = pca.fit_transform(norm_hog_val)

In [ ]:
np.save(arraypath+'pca_hog_val.npy', pca_hog_val)

In [ ]:
norm_hog_val=''
pca_hog_val=''

In [ ]:
norm_hog_test=np.load(arraypath+'norm_hog_test.npy')

In [ ]:
pca_hog_test = pca.fit_transform(norm_hog_test)

In [ ]:
np.save(arraypath+'pca_hog_test.npy', pca_hog_test)

In [ ]:
norm_hog_test=''
pca_hog_test=''

#### LBP

In [ ]:
norm_lbp_train, norm_lbp_val,norm_lbp_test = norm_features_minmax2(lbp_train, lbp_val,lbp_test)

In [ ]:
np.save(arraypath+'norm_lbp_train.npy', norm_lbp_train)
np.save(arraypath+'norm_lbp_val.npy', norm_lbp_val)
np.save(arraypath+'norm_lbp_test.npy', norm_lbp_test)

In [ ]:
norm_lbp_train=''
norm_lbp_val=''
norm_lbp_test=''

In [ ]:
norm_lbp_train = np.load(arraypath+'norm_lbp_train.npy')

In [ ]:
pca = PCA(n_components=100)
pca_lbp_train = pca.fit_transform(norm_lbp_train)

In [ ]:
np.save(arraypath+'pca_lbp_train.npy', pca_lbp_train)

In [ ]:
pca_lbp_train=''
norm_lbp_train=''

In [ ]:
norm_lbp_val = np.load(arraypath+'norm_lbp_val.npy')

In [ ]:
pca_lbp_val = pca.fit_transform(norm_lbp_val)

In [ ]:
np.save(arraypath+'pca_lbp_val.npy', pca_lbp_val)

In [ ]:
pca_lbp_val=''
norm_lbp_val=''

In [ ]:
norm_lbp_test = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/norm_lbp_test.npy')

In [ ]:
pca_lbp_test = pca.fit_transform(norm_lbp_test)

In [ ]:
np.save(arraypath+'pca_lbp_test.npy', pca_lbp_test)

In [ ]:
pca_lbp_test=''
norm_lbp_test=''

In [ ]:
pca_lbp_train = np.load(arraypath+'pca_lbp_train.npy')
pca_lbp_val = np.load(arraypath+'pca_lbp_val.npy')
pca_lbp_test = np.load(arraypath+'pca_lbp_test.npy')

In [ ]:
print(np.shape(pca_lbp_train),np.shape(pca_lbp_val))

#### SIFT

In [ ]:
norm_sift_train, norm_sift_val,norm_sift_test = norm_features_minmax2(sift_train, sift_val,sift_test)

In [ ]:
np.save(arraypath+'norm_sift_train.npy', norm_sift_train)
np.save(arraypath+'norm_sift_val.npy', norm_sift_val)
np.save(arraypath+'norm_sift_test.npy', norm_sift_test)

In [ ]:
pca = PCA(n_components=100)
pca_sift_train = pca.fit_transform(norm_sift_train)
pca_sift_val = pca.transform(norm_sift_val)
pca_sift_test = pca.transform(norm_sift_test)

In [ ]:
np.save(arraypath+'pca_sift_train.npy', pca_sift_train)
np.save(arraypath+'pca_sift_val.npy', pca_sift_val)
np.save(arraypath+'pca_sift_test.npy', pca_sift_test)

In [ ]:
pca_sift_train = np.load(arraypath+'pca_sift_train.npy')
pca_sift_val = np.load(arraypath+'pca_sift_val.npy')
pca_sift_test = np.load(arraypath+'pca_sift_test.npy')

#### Gray

In [ ]:
norm_gray_train, norm_gray_val,norm_gray_test = norm_features_minmax2(flat_gray_train, flat_gray_val,flat_gray_test)

In [ ]:
pca = PCA(n_components=100)
pca_gray_train = pca.fit_transform(norm_gray_train)
pca_gray_val = pca.transform(norm_gray_val)
pca_gray_test = pca.transform(norm_gray_test)

In [ ]:
np.save(arraypath+'pca_gray_train.npy', pca_gray_train)
np.save(arraypath+'pca_gray_val.npy', pca_gray_val)
np.save(arraypath+'pca_gray_test.npy', pca_gray_test)

In [ ]:
pca_gray_train=''
pca_gray_val=''
pca_gray_test=''

In [ ]:
pca_gray_train = np.load(arraypath+'pca_gray_train.npy')
pca_gray_val = np.load(arraypath+'pca_gray_val.npy')
pca_gray_test = np.load(arraypath+'pca_gray_test.npy')

#### KAZE

In [ ]:
norm_kaze_train, norm_kaze_val, norm_kaze_test = norm_features_minmax2(kaze_train, kaze_val,kaze_test)

In [ ]:
np.save(arraypath+'norm_kaze_train.npy', norm_kaze_train)
np.save(arraypath+'norm_kaze_val.npy', norm_kaze_val)
np.save(arraypath+'norm_kaze_test.npy', norm_kaze_test)

In [ ]:
norm_kaze_train.shape, norm_kaze_val.shape, norm_kaze_test.shape

In [ ]:
pca = PCA(n_components=100)
pca_kaze_train = pca.fit_transform(norm_kaze_train)
pca_kaze_val = pca.transform(norm_kaze_val)
pca_kaze_test = pca.transform(norm_kaze_test)

In [ ]:
np.save(arraypath+'pca_kaze_train.npy', pca_kaze_train)
np.save(arraypath+'pca_kaze_val.npy', pca_kaze_val)
np.save(arraypath+'pca_kaze_test.npy', pca_kaze_test)

In [ ]:
pca_kaze_train =np.load(arraypath+'pca_kaze_train.npy')
pca_kaze_val = np.load(arraypath+'pca_kaze_val.npy')
pca_kaze_test = np.load(arraypath+'pca_kaze_test.npy')

In [ ]:
pca_kaze_train.shape, pca_kaze_val.shape, pca_kaze_test.shape

#### Color Histogram

In [ ]:
norm_hist_train, norm_hist_val, norm_hist_test = norm_features_minmax2(hist_train, hist_val, hist_test)

In [ ]:
np.save(arraypath+'norm_hist_train.npy', norm_hist_train)
np.save(arraypath+'norm_hist_val.npy', norm_hist_val)
np.save(arraypath+'norm_hist_test.npy', norm_hist_test)

In [ ]:
from sklearn.decomposition import PCA
pca_1 = PCA(n_components=100)
pca_color_hist_train = pca_1.fit_transform(norm_hist_train)
pca_color_hist_val = pca_1.transform(norm_hist_val)
pca_color_hist_test = pca_1.transform(norm_hist_test)

In [ ]:
np.save(arraypath+'pca_color_hist_train.npy', pca_color_hist_train)
np.save(arraypath+'pca_color_hist_val.npy', pca_color_hist_val)
np.save(arraypath+'pca_color_hist_test.npy', pca_color_hist_test)

In [ ]:
pca_color_hist_train = np.load(arraypath+'pca_color_hist_train.npy')
pca_color_hist_val = np.load(arraypath+'pca_color_hist_val.npy')
pca_color_hist_test = np.load(arraypath+'pca_color_hist_test.npy')

### **Reduce Features by LDA**

#### HOG

In [ ]:
norm_hog_train=np.load(arraypath+'norm_hog_train.npy')

In [ ]:
lda = LDA()
lda_pca_train = lda.fit_transform(norm_hog_train, train_y)

In [ ]:
norm_hog_train=''

In [ ]:
norm_hog_val=np.load(arraypath+'norm_hog_val.npy')

In [ ]:
lda_pca_val = lda.transform(norm_hog_val)

In [ ]:
norm_hog_val=''

In [ ]:
norm_hog_test=np.load(arraypath+'norm_hog_test.npy')

In [ ]:
lda_pca_test = lda.transform(norm_hog_test)

In [ ]:
norm_hog_test=''

#### SIFT

In [ ]:
norm_surf_train=np.load(arraypath+'norm_sift_train.npy')
norm_surf_val=np.load(arraypath+'norm_sift_val.npy')
norm_surf_test=np.load(arraypath+'norm_sift_test.npy')

In [ ]:
lda = LDA()
lda_sift_train = lda.fit_transform(norm_surf_train, train_y)
lda_sift_val = lda.transform(norm_surf_val)
lda_sift_test = lda.transform(norm_surf_test)

In [ ]:
np.save(arraypath+'lda_sift_train.npy', lda_sift_train)
np.save(arraypath+'lda_sift_val.npy', lda_sift_val)
np.save(arraypath+'lda_sift_test.npy', lda_sift_test)

In [ ]:
lda_sift_train = np.load(arraypath+'lda_sift_train.npy')
lda_sift_val = np.load(arraypath+'lda_sift_val.npy')
lda_sift_test = np.load(arraypath+'lda_sift_test.npy')

####  Color Histogram

In [ ]:
norm_hist_train=np.load(arraypath+'norm_hist_train.npy' )
norm_hist_val=np.load(arraypath+'norm_hist_val.npy' )
norm_hist_test=np.load(arraypath+'norm_hist_test.npy' )

In [ ]:
lda = LDA()
lda_hist_train = lda.fit_transform(norm_hist_train, train_y)
lda_hist_val = lda.transform(norm_hist_val)
lda_hist_test = lda.transform(norm_hist_test)

In [ ]:
np.save(arraypath+'lda_hist_train.npy', lda_hist_train)
np.save(arraypath+'lda_hist_val.npy', lda_hist_val)
np.save(arraypath+'lda_hist_test.npy', lda_hist_test)

In [ ]:
lda_hist_train = np.load(arraypath+'lda_hist_train.npy')
lda_hist_val = np.load(arraypath+'lda_hist_val.npy')
lda_hist_test =  np.load(arraypath+'lda_hist_test.npy')

In [ ]:
lda_hist_train.shape, lda_hist_val.shape, lda_hist_test.shape

####  KAZE

In [ ]:
norm_kaze_train=np.load(arraypath+'norm_kaze_train.npy' )
norm_kaze_val=np.load(arraypath+'norm_kaze_val.npy')
norm_kaze_test=np.load(arraypath+'norm_kaze_test.npy' )

In [ ]:
lda = LDA()
lda_kaze_train = lda.fit_transform(norm_kaze_train,train_y)
lda_kaze_val = lda.transform(norm_kaze_val)
lda_kaze_test = lda.transform(norm_kaze_test)

In [ ]:
np.save(arraypath+'lda_kaze_train.npy', lda_kaze_train)
np.save(arraypath+'lda_kaze_val.npy', lda_kaze_val)
np.save(arraypath+'lda_kaze_test.npy', lda_kaze_test)

In [ ]:
lda_kaze_train = np.load(arraypath+'lda_kaze_train.npy')
lda_kaze_val = np.load(arraypath+'lda_kaze_val.npy')
lda_kaze_test = np.load(arraypath+'lda_kaze_test.npy')

In [ ]:
lda_kaze_train.shape, lda_kaze_val.shape, lda_kaze_test.shape

### **Combine Features**

In [ ]:
hog_train = np.load(arraypath+'hog_train.npy')
hog_val = np.load(arraypath+'hog_val.npy')

kaze_train = np.load(arraypath+'kaze_train.npy')
kaze_val = np.load(arraypath+'kaze_val.npy')

hist_train = np.load(arraypath+'hist_train.npy')
hist_val = np.load(arraypath+'hist_val.npy')

sift_train = np.load(arraypath+'sift_train.npy')
sift_val = np.load(arraypath+'sift_val.npy')

In [ ]:
features_train = None
features_val = None

for t, v in zip([hog_train, hist_train, sift_train, kaze_train],
                   [hog_val, hist_val, sift_val, kaze_val]):
  if features_train is None:
    features_train = t
    features_val = v
  else:
    features_train = combine_features([features_train, t])
    features_val = combine_features([features_val, v])

In [ ]:
np.save(arraypath+'features_train.npy', features_train)
np.save(arraypath+'features_val.npy', features_val)


### **Combine Features for PCA**

In [ ]:
pca_hog_train = np.load(arraypath+'pca_hog_train.npy')
pca_hog_val = np.load(arraypath+'pca_hog_val.npy')

pca_kaze_train = np.load(arraypath+'pca_kaze_train.npy')
pca_kaze_val = np.load(arraypath+'pca_kaze_val.npy')

pca_color_hist_train = np.load(arraypath+'pca_color_hist_train.npy')
pca_color_hist_val = np.load(arraypath+'pca_color_hist_val.npy')

pca_sift_train = np.load(arraypath+'pca_sift_train.npy')
pca_sift_val = np.load(arraypath+'pca_sift_val.npy')

pca_lbp_train = np.load(arraypath+'pca_lbp_train.npy')
pca_lbp_val = np.load(arraypath+'pca_lbp_val.npy')

In [ ]:
features_train_pca = None
features_val_pca = None

for t, v in zip([pca_hog_train, pca_color_hist_train, pca_sift_train,  pca_lbp_train, pca_kaze_train],
                   [pca_hog_val, pca_color_hist_val, pca_sift_val, pca_lbp_val, pca_kaze_val]):
  if features_train_pca is None:
    features_train_pca = t
    features_val_pca = v
  else:
    features_train_pca = combine_features([features_train_pca, t])
    features_val_pca = combine_features([features_val_pca, v])

In [ ]:
np.save(arraypath+'features_train_pca.npy', features_train_pca)
np.save(arraypath+'features_val_pca.npy', features_val_pca)

In [ ]:
features_train_pca = np.load(arraypath+'features_train_pca.npy')
features_val_pca = np.load(arraypath+'features_val_pca.npy')

### **Combine Features for LDA**

In [ ]:
lda_kaze_train = np.load(arraypath+'lda_kaze_train.npy')
lda_kaze_val = np.load(arraypath+'lda_kaze_val.npy')

lda_hist_train = np.load(arraypath+'lda_hist_train.npy')
lda_hist_val = np.load(arraypath+'lda_hist_val.npy')

lda_sift_train = np.load(arraypath+'lda_sift_train.npy')
lda_sift_val = np.load(arraypath+'lda_sift_val.npy')

In [ ]:
features_train_lda = None
features_val_lda = None

for t, v in zip([lda_hist_train, lda_sift_train, lda_kaze_train],
                   [lda_hist_val, lda_sift_val, lda_kaze_val]):
  if features_train_lda is None:
    features_train_lda = t
    features_val_lda = v
  else:
    features_train_lda = combine_features([features_train_lda, t])
    features_val_lda = combine_features([features_val_lda, v])

In [ ]:
np.save(arraypath+'features_train_lda.npy', features_train_lda)
np.save(arraypath+'features_val_lda.npy', features_val_lda)

In [ ]:
features_train_lda = np.load(arraypath+'features_train_lda.npy')
features_val_lda = np.load(arraypath+'features_val_lda.npy')

### **Reduce Features by LDA on PCA**

In [ ]:
lda = LDA()
features_train_pca = np.load(arraypath+'features_train_pca.npy')
features_val_pca = np.load(arraypath+'features_val_pca.npy')
lda_pca_train = lda.fit_transform(features_train_pca, train_y)
lda_pca_val = lda.transform(features_val_pca)
np.save(arraypath+'lda_pca_train.npy', lda_pca_train)
np.save(arraypath+'lda_pca_val.npy', lda_pca_val)

In [ ]:
lda = LDA()
features_train_pca = np.load(arraypath+'features_train_pca.npy')
features_test_pca = np.load(arraypath+'features_test_pca.npy')
lda_pca_train = lda.fit_transform(features_train_pca, train_y)
lda_pca_test = lda.transform(features_test_pca)
np.save(arraypath+'lda_pca_test.npy', lda_pca_test)